In [6]:
%pip show geopandas

Name: geopandas
Version: 1.0.1
Summary: Geographic pandas extensions
Home-page: 
Author: 
Author-email: Kelsey Jordahl <kjordahl@alum.mit.edu>
License: BSD 3-Clause
Location: c:\Users\user\anaconda3\Lib\site-packages
Requires: numpy, packaging, pandas, pyogrio, pyproj, shapely
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [7]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# 1. 지역명을 광역시/도로 변환하는 함수
def convert_region(name):
    if name == '전국':
        return None
    parts = name.split()
    region_map = {
        '서울': '서울특별시', '부산': '부산광역시', '대구': '대구광역시',
        '인천': '인천광역시', '광주': '광주광역시', '대전': '대전광역시',
        '울산': '울산광역시', '세종': '세종특별자치시', '경기': '경기도',
        '강원': '강원도', '충북': '충청북도', '충남': '충청남도',
        '전북': '전라북도', '전남': '전라남도', '경북': '경상북도',
        '경남': '경상남도', '제주': '제주특별자치도'
    }
    return region_map.get(parts[0], None)

# 2. 필터된 데이터프레임에 광역시도 열 추가
filtered_df = df[df['지역'] != '서울전체'] 
filtered_df['광역시도'] = filtered_df['지역'].apply(convert_region)

# 3. 지역별 공고 수 집계
region_counts = filtered_df['광역시도'].value_counts().reset_index()
region_counts.columns = ['지역', '공고수']

# 4. GeoJSON 파일 불러오기
gdf = gpd.read_file('../data/skorea_provinces_geo_simple.json')

# 5. 병합 (left join으로 모든 지역 유지)
merged = gdf.merge(region_counts, left_on='name', right_on='지역', how='left')
merged['공고수'] = merged['공고수'].fillna(0).astype(int)

# 6. 일부 지역의 라벨이 겹치지 않도록 위치 보정값 정의
label_offsets = {
    '서울특별시': (0, 0.1),
    '세종특별자치시': (0, -0.2),
    '인천광역시': (-0.3, 0.1),
    '대전광역시': (0.1, -0.2),
    '광주광역시': (-0.2, -0.2),
    '대구광역시': (0.2, -0.1),
    '부산광역시': (0.3, -0.3),
    '울산광역시': (0.2, -0.2),
}

# 7. 시각화
fig, ax = plt.subplots(1, 1, figsize=(10, 12))
merged.plot(
    column='공고수',
    cmap='Blues',
    linewidth=0.8,
    ax=ax,
    edgecolor='0.8',
    legend=True,
    legend_kwds={'shrink': 0.6, 'label': '공고 수'}
)

# 8. 지도 위에 텍스트 표시 (지역명 + 공고수)
for idx, row in merged.iterrows():
    point = row.geometry.representative_point()
    dx, dy = label_offsets.get(row['name'], (0, 0))
    plt.annotate(
        text=f"{row['name']}\n{row['공고수']}건",
        xy=(point.x + dx, point.y + dy),
        ha='center',
        fontsize=8,
        color='black'
    )

plt.title('지역별 채용공고 수 (광역시도 기준)', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()


NameError: name 'df' is not defined